In [1]:
import socket
import os
from _thread import *
from random import randrange, getrandbits
from math import gcd
import json
import time

In [2]:
def is_prime(n, k=8):
    """ Test if a number is prime
        Args:
            n -- int -- the number to test
            k -- int -- the number of tests to do
        return True if n is prime
    """
    # Test if n is not even.
    # But care, 2 is prime !
    if n == 2 or n == 3:
        return True
    if n <= 1 or n % 2 == 0:
        return False
    # find r and s
    s = 0
    r = n - 1
    while r & 1 == 0:
        s += 1
        r //= 2
    # do k tests
    for _ in range(k):
        a = randrange(2, n - 1)
        x = pow(a, r, n)
        if x != 1 and x != n - 1:
            j = 1
            while j < s and x != n - 1:
                x = pow(x, 2, n)
                if x == 1:
                    return False
                j += 1
            if x != n - 1:
                return False
    return True

def generate_prime_candidate(length):
    """ Generate an odd integer randomly
        Args:
            length -- int -- the length of the number to generate, in bits
        return a integer
    """
    # generate random bits
    p = getrandbits(length)
    # apply a mask to set MSB and LSB to 1
    p |= (1 << length - 1) | 1
    return p
def generate_prime_number(prev, length=8):
    """ Generate a prime
        Args:
            length -- int -- length of the prime to generate, in          bits
        return a prime
    """
    p = 4
    # keep generating while the primality test fail
    while not is_prime(p, 4) or p == prev:
        p = generate_prime_candidate(length)
    return p


In [3]:
def generate_public_key(phi):
    e = 2
    while True:
        if gcd(e, phi) == 1:
            return e
        e += 1

In [4]:
def generate_key():
    p = generate_prime_number(1,8)
    q = generate_prime_number(p,8)
    print(p,q)
    n = p * q
    phi = (p-1) * (q-1)
    e = generate_public_key(phi)
    d = pow(e, -1, phi)
    return (e, n), (d, n)

In [5]:
def encrypt(pk, plaintext):
    # Unpack the key into it's components
    key, n = pk
    # Convert each letter in the plaintext to numbers based on the character using a^b mod m
    cipher = [pow(ord(char), key, n) for char in plaintext]
    # Return the array of bytes
    return cipher

In [6]:
def decrypt(pk, ciphertext):
    # Unpack the key into its components
    key, n = pk
    # Generate the plaintext based on the ciphertext and key using a^b mod m
    plain = [chr(pow(char, key, n)) for char in ciphertext]
    # Return the array of bytes as a string
    return ''.join(plain)

In [7]:
def get_timestamp():
    return int(time.time())

def get_nonce():
    return randrange(2**16)

def get_nonce_acknowledgment(nonce_received):
    # XORing with a constant value twice gives back the original value
    return nonce_received ^ 0xFFFF 

def get_validity_timestamp():
    return get_timestamp() + 5*60*1000

def check_validity_message(valid_till):
    return valid_till > get_timestamp()

In [8]:
def add_client_info(client_name, public_key, client_info_dict, client_id, host_address, port_number):
    client_info_dict[client_name] = {
        "public_key": public_key,
        "client_id": client_id,
        "host_address": host_address,
        "port_number": port_number
    }

def get_client_info(client_name, client_info_dict):
    return client_info_dict[client_name] if client_name in client_info_dict else None

In [9]:
def create_message(message_type, message_payload, input_client_name, client_id):
    return {
        'type': message_type,
        'client_name':input_client_name,
        'id': client_id,
        'timestamp': get_timestamp(),
        'valid_till':get_validity_timestamp(),
        'nonce':get_nonce(),
        'message_payload': message_payload
    }

In [10]:
ServerSideSocket = socket.socket()
host = '127.0.0.1'
port = 3000
ThreadCount = 0
public_key, private_key = generate_key()
print("Public Key: ", public_key)
print("Private Key: ", private_key)
public_key_ring = {}
## public_key_ring structure
# {
#     'client_name': { # unique string name of the client
#         'host_address': host_address, # host address of the client
#         'port_number': port, # port of the client to connect to
#         'public_key': public_key, # public key of the client
#         'client_id': client_id # unique 16 byte id of the client
#     }
# }

message_nonce_dict = {}
## message_nonce_dict structure 
# {
#     'client_name': 'nonce',
# }

try:
    ServerSideSocket.bind((host, port))
except socket.error as e:
    print(str(e))
print('Socket is listening..')
ServerSideSocket.listen(5)

## message structure
# {
#     "type":'handshake', # type of message
#     'id':'', # unique id of the 
#     'client_name': '', # name of the client
#     'public_key': '', # public key of the sender
#     'timestamp': '', # timestamp of the message
#     'nonce': '', # nonce of the message
#     'valid_till': '', # validity timestamp of the message
#     'acknowledgement_nonce': '' # f(previous_message_nonce) XORed with a constant value
# }

def multi_threaded_client(connection, address):
    client_handshake_message_str = connection.recv(2048).decode('utf-8')
    client_handshake_message = json.loads(client_handshake_message_str)
    if not check_validity_message(client_handshake_message['valid_till']):
        print('Invalid message, message expired')
        return
    else:
        print('Validity check passed')
        client_name = client_handshake_message['client_name']
        client_id = client_handshake_message['id']
        client_public_key = client_handshake_message['message_payload']['public_key']
        nonce_received = client_handshake_message['nonce']
        client_host_address = client_handshake_message['message_payload']['client_host_address']
        client_port_number = client_handshake_message['message_payload']['client_port_number']
        add_client_info(client_name,client_public_key, public_key_ring, client_id, client_host_address, client_port_number)

        message_payload = {
            'public_key':public_key,
            'acknowledgement_nonce': get_nonce_acknowledgment(nonce_received)
        }
        server_handshake_message = create_message('handshake',message_payload, 'pkda', 'public key distribution authority')
        server_handshake_message_str = json.dumps(server_handshake_message)
        connection.sendall(str.encode(server_handshake_message_str))
        while True:
            client_request_message_str = connection.recv(2048).decode('utf-8')
            if not client_request_message_str:
                break
            else:
                client_request_message = json.loads(client_request_message_str)
                sender_client_name = client_request_message['client_name']
                requested_client_name = client_request_message['message_payload']['requested_client']
                if not check_validity_message(client_request_message['valid_till']):
                    print('Invalid message, message expired')
                    break
                else:
                    print('Public key request received from client: {} for client {} '.format(sender_client_name, requested_client_name))
                    requested_client_info = get_client_info(requested_client_name, public_key_ring)
                    message_payload = {
                        'request': client_request_message,
                        'requested_client_info': requested_client_info,
                        'acknowledgement_nonce': get_nonce_acknowledgment(client_request_message['nonce'])
                    }
                    server_response_message = create_message('public_key_request_response',message_payload, 'pkda' ,'public key distribution authority',)
                    server_response_message_str = json.dumps(server_response_message)
                    encrypted_message = encrypt(private_key, server_response_message_str)
                    connection.sendall(str.encode(json.dumps(encrypted_message)))
    connection.close()

while True:
    Client, address = ServerSideSocket.accept()
    print('Connected to: ' + address[0] + ':' + str(address[1]))
    start_new_thread(multi_threaded_client, (Client, address))
    ThreadCount += 1
    print('Thread Number: ' + str(ThreadCount))
ServerSideSocket.close()

149 163
Public Key:  (5, 24287)
Private Key:  (19181, 24287)
Socket is listening..
Connected to: 127.0.0.1:51401
Thread Number: 1
Validity check passed
Connected to: 127.0.0.1:51417
Thread Number: 2
Validity check passed
Public key request received from client: client2 for client client1 
Public key request received from client: client1 for client client2 


: 

: 